# Set kernel to: `py-pythonEnv`
If this kernel is not available, see the Kernel Installation Instructions section located in at the [bottom](#apdx)  of this Notebook.

In [ ]:
import pandas as pd
# view available ActionRuntimes
pd.DataFrame(c3.MetadataStore.tag().runtimes().toJson()).T

In [ ]:
# get just the Python runtimes
runtimes = pd.DataFrame(c3.MetadataStore.tag().runtimes().toJson()).T
runtimes[runtimes["language"] == "Python"]

# Import libraries used in this notebook:

In [ ]:
from datetime import datetime
import pandas as pd

# Table of Contents 

* [C3 Python Basics](#1)
    * [Fetch and Filter in Python](#1.1)
        * [A. Common Comands](#1.1A)
        * [B. View Type Dcumentation](#1.1B)
        * [C. Fetch Method in Python](#1.1C)
        * [D. Evaluate Method in Python](#1.1D)
        * [E. Other Useful Operations](#1.1E)
    * [CRUD operations in Python](#1.2)
    * [Prototype Methods in Python](#1.3)
        * [A. Write code](#1.3A)
        * [A. Test code](#1.3B)
    * [AsyncActions](#1.4)

[Appendix - Kernel Installation Instructions](#apdx)

# C3 Python Basics <a class="anchor" id="1">

We can access the C3 Type system and the data provisioned on a given Tenant and Tag using the C3 Python SDK.

In [ ]:
c3

If you are running this notebook within the training environments on your tag, the c3 variable connection is already set up for you. If you are running the notebook locally and would like to connect remotely to the c3 environment, you will have to configure this variable yourself using [c3Remote](https://developer.c3.ai/docs/7.28.0/guide/guide-ml-ds/ds-c3-remote).

## 1. Fetching and Filtering Data in Python <a class="anchor" id="1.1">

### A. Comon Commands <a class="anchor" id="1.1A">
Some useful operations/commands for Jupyter Notebooks and the JS Console.

#### Generic Type System Calls:
-  `c3.Type.method(...)`

| Use Case | Jupyter Notebook Command | JS Console Command |
| --- | --- | --- |
|View Documentation on `MyType` | help(c3.MyType) | c3ShowType(MyType) |
    |View Documentation on a `MyType` method | help(c3.MyType.myMethod) | N/A |
| Find the number `MyType` records| c3.MyType.fetchCount() | c3Count(MyType) |
| Fetch `MyType` records| c3.MyType.fetch()| MyType.fetch() |
| Fetch `MyType` records in a table| pd.DataFrame(c3.MyType.fetch().objs.toJson()) ${^\dagger}$ | c3Grid(MyType.fetch())|
| Fetch `MyType` records in a table with filters | c3.MyType.fetch(spec={'filter':" ... "})${^\ddagger}$ | c3Grid(MyType.fetch({filter: " ... "})) |

> ${^\dagger}$ - Instead of using a C3 function, you can use a `pandas` `DataFrame`.

> ${^\ddagger}$ - Notice the extra single-quote around `filter`, and assigning the variable `spec` in the JSON-object, this is the `FetchSpec`. Run `help(c3.FetchSpec)` in Jupyter for more details.     

### B. View Type Dcumentation:<a class="anchor" id="1.1B">

We can view Type documentation directly in our Jupyter notebook (as well as in the console as you saw earlier).    

Use `help(c3.MyType)` to see Type Documentation.

In [ ]:
help(c3.FetchSpec)

In [ ]:
help(c3.MyEntityType) # view the documentation on an entity Type of your choice from the Entity Relation Diagram

#### View Method Documentation:
Use `help(c3.MyType.myMethod)` to view documentation for a specific method on a Type.

In [ ]:
help(c3.Persistable.fetch)

### C. Fetch Method in Python <a class='anchor' id='1.1C'>

To view what records exist for a given Type, we can execute a `.fetch()` query on entity (Persistable aka Entity) Types. Why Persistable/Entity Types? Because by definition, those are the Types that have a datatable attached to them.

#### Simple Fetch:
Use `c3.MyType.fetch()` to get data with the standard fields

In [ ]:
c3.City.fetch()

In [ ]:
c3.MyChosenEntityType.fetch()# fetch another entity Type of your choice from the Entity Relation Diagram

#### Fetch and Dislpay in a Pandas DataFrame:
Use `.objs.toJson()` to get a fetch result in an array for a json and wrap that with `pd.DataFrame` to plot a table as a pandas dataframe.

In [ ]:
myFetchResult = c3.City.fetch()

pd.DataFrame(myFetchResult.objs.toJson())

#### Fetch Count:
There are additional fetch-related methods, such as `c3.MyType.fetchCount()`, that you can explore in the documentation.

In [ ]:
c3.City.fetchCount()

#### Fancy Fetches:
You can refine your fetch query to only return certain records by using a fetchSpec. This allows you, for example, to limit the number of results, filter the data to be returned, or order the data by a particular column (see `help(c3.FetchSpec)` to read about more options).

In [ ]:
help(c3.ExpressionEngineFunction)

In [ ]:
myFetchResult = c3.City.fetch(spec={
#                                       'limit': 2,
#                                       'include': 'latitude',
#                                       'include': 'buildings',
#                                       'order': "ascending(latitude)",
                                      'filter': "startsWith(name, 'M')"
                                 })

pd.DataFrame(myFetchResult.objs.toJson())

### Use This Space for "CHALLENGE: Fetch and Filter in Jupyter"
    
    Add additional cells as needed.

### D. Evaluate Method in Python <a class="anchor" id="1.1D">

The `evaluate` method can be used similarly to a fetch. It returns tuples, so in order to view the results, you will have to access the tuples instead of the objs.

Use `c3.MyType.evaluate(spec={'projection':"myField, myFunction"}).tuples`


In [ ]:
c3.City.evaluate(spec = {
                             'projection':"name, latitude, longitude",
#                              'filter': "id == 'RWC'",#" && myOtherField == 'myOtherString'",
#                              'group':"name",
                             "order":"ascending(name)",
                             #'limit': 10,
                             #'offset': 5
                        }).tuples

Additionally, with `evaluate`, you can navigate between Types when fetching data, so that you can pull data from more distant (i.e. not directly connected) Types. Compare using a `fetch` to using `evaluate`:

In [ ]:
viewBuildingsInTheCity = c3.City.fetch(spec={
#                                       'limit': -1,
                                      'include': 'buildings',
#                                       'order': "ascending(latitude)",
#                                       'filter': "contains(name, 'e')"
                                 })

pd.DataFrame(viewBuildingsInTheCity.objs.toJson())

In [ ]:
viewApartmentsInTheCity = c3.City.evaluate(spec={
                                                'projection': 'buildings.apartments',
                                                'filter': "contains(name, 'San')"
                                 })

viewApartmentsInTheCity.tuples

We've written a helper function for you to convert these tuples to a pandas dataframe.

In [ ]:
def evaluate_projection(c3_type, projection_dict, column_name = [], ignore_ref = True):
    """
    evaluate_projection(c3_type, projection_dict, column_name = [])
    evalaute_projection_data unpack data from evalaute projection to pandas dataframe
    parameters
    ----------
    c3_type : c3.Type
        Type to evalaute projection
    projection_dict : dict
        A dictionary contain projection details 
    column_name : [string], default = []
        A list of string contains column name for projection. If not specified, 
        the function extract columns name from column header. 
    ignore_ref : bool, default = True
        if ignore_ref is True, the return data frame will unpack the reference column. In other words,
        instead of getting c3.PhysicalAsset(id: "abcd"), you will get "abcd"
    returns
    -------
    df : pandas dataframe
        output of evalaute projection 
    examples
    --------
    >>> projection_dict = {'projection': "assetId, count())",
                             'group': 'assetId',
                             'filter': "startsWith(workOrderType, '{}')".format('correction')}
    >>> c3_type = c3.WorkOrder 
    >>> evaluate_projection(c3_type, projection_dict)
        assetId    count()
    0   ABHDG        5
    1   UUYSH101B   10
    """
    import pandas as pd
    import warnings 
    import re
    #Input check#
    if not isinstance(c3_type, c3.Type):
        raise ValueError("The input c3_type is not c3.Type (e.g., c3.PhysicalAsset)")
    if len(column_name) != 0:
        nCol = len(projection_dict['projection'].split(","))
        if nCol != len(column_name):
            raise ValueError("Number of column names in column_name are not equal to number of projections")
    if not "projection" in projection_dict:
        raise ValueError("projection_dict must contain 'projection' as one of the key")
    #### Evaluate projection ####
    data = c3_type.evaluate(projection_dict)
    if data.count == 0:
        warnings.warn("No output from evalaute projection")
        return pd.DataFrame([])
    N = len(data.tuples[0].cells)          #number of projection's columns 
    #one line of unpack data 
    df = pd.DataFrame([[x.cells[i].value() for x in data.tuples] for i in range(N)]).transpose()
    #unpack reference columns (Forien key column)
    if ignore_ref == True:
        for i in range(N):
            if "c3." in str(type(df.iloc[0, i])):
                df[i] = df[i].apply(lambda x: [list(x) for x in zip(*x.__dict__.items())][-1][0])    
    #Renaming column 
    if len(column_name) == 0:
        r = re.compile(r'(?:[^,(]|\([^)]*\))+')
        column_name = r.findall(projection_dict['projection'])
    rename_dict = {k: v for (k, v) in zip(range(N), column_name)}
    df.rename(rename_dict, axis = 1, inplace = True) 
    return df

In [ ]:
projection_dict = {
                    'projection': 'buildings.apartments',
                    'filter': "contains(name, 'San')"
                  }
c3_type = c3.City

In [ ]:
evaluate_projection(c3_type, projection_dict)

In addition, queries with `evaluate` can also be used to show the results of transformations, e.g. mathematical operations, on a field.

In [ ]:
projection_dict = {
                         'projection':"name, latitude, cos(latitude), longitude, 3*longitude",
#                          'filter': "contains(name,'a')",
                         #'group':"myField",
#                          "order":"ascending(name)",
                         #'limit': 10,
                         #'offset': 5
                  }
c3_type = c3.City

In [ ]:
evaluate_projection(c3_type, projection_dict)

**WARNING** `evaluate` displays some odd behavior when returning projected nested arrays. If the results contain multiple "levels" (e.g one City [top level] has many Buildings [nested level], which have many Apartments [nested-nested-level]), then the projection will "cut off" at the top level. Compare the following `evaluation`s:

In [ ]:
projection_dict = {
                    'projection': 'name, buildings, buildings.apartments',
#                     'filter': "contains(name, 'e')"
                  }
c3_type = c3.City

In [ ]:
evaluate_projection(c3_type, projection_dict)
# only returning the "top level", so the first Building in the City and the first Apartment in that Building

In [ ]:
projection_dict = {
                    'projection': 'buildings.apartments',
#                     'filter': "contains(name, 'e')"
                  }
c3_type = c3.City

In [ ]:
evaluate_projection(c3_type, projection_dict)
# returning all of the Apartments that are in a City (all of them)

### Use This Space for "CHALLENGE: Evaluate Method in Jupyter" 
    
    Add additional cells as needed.
    
    Hint: You may want to run `help(c3.MyEntityType)` or use the ERD if you're not sure what the fields on a Type
    are called!

In [ ]:
help(c3.SmartBulbMeasurement)

In [ ]:
projection_dict = {
                    "projection": "field1, field2"
                  }
c3_type = c3.MyType

evaluate_projection(c3_type, projection_dict)

## 2. CRUD Operations in Python <a class="anchor" id="1.2">

We have seen how to fetch data records and how to view those results in a pandas DataFrame

In [ ]:
# c3.MyEntityType.fetch()
pd.DataFrame(c3.MyType.fetch().objs.toJson())

As a Data Scientist, it's unlikely you'll be creating or editing individual records by hand often; however, it's important to understand how the C3 platform manipulates records and data for when these operations are integrated into platform or custom methods later.

##### Create Method

An entity (Persistable) type only requires a unique id when creating a new record. We can create new records in this type using the `.create()` method and passing it an UpsertSpec. If no unique id is specified, one is created.

In [ ]:
c3.MyType.create(this={
                         'id':'UniqueId1',
                         'name': 'Name1'
                      })


We can now view the record that we just created using the `.get()` method and passing it the record id.

In [ ]:
MyEntityInstance = c3.MyType.get('UniqueId1')

In [ ]:
MyEntityInstance.id

##### What's the difference between `get()` and `fetch()`?

`fetch()` returns a set of records from a datatable and does not require any parameters (although you *can* include a fetch spec). `get()` requires an id as a parameter and will then return the data for only that specified record/the specific instance.

##### Merge Method

If we want to make changes to an existing record and to only update certain fields on a created record, we can use the `merge()` method

In [ ]:
MyEntityInstance.latitude = 32.7
MyEntityInstance.longitude = None

In [ ]:
MyEntityInstance.merge()

Note that you do not have to use the `get()` method to make updates to records, but rather you can pass in a MergeSpec and simply merge new fields directly:

In [ ]:
c3.MyType.merge(this={
                             'id': 'UniqueId1',
                             'name': 'Name1',
                             'latitude': 32.7,
                             'longitude': -101.1
                   })

In [ ]:
pd.DataFrame(c3.MyType.fetch().objs.toJson())

##### Upsert and Update Methods

There are 2 additional methods available to create or edit existing records: `upsert()` and `update()`. Their syntax is the same as that for `create()` and `merge()`. How they work:

* `upsert()` will update the record if it exists or create it if it doesn't. If you do not specify the id, `upsert()` will create a new record with a new id.
* `update()` requires the record to already exist and requires an id to be passed in or it will throw an error.

Both methods require you to specify values for all required fields in the upsert/update spec or they will throw an error.

**BEWARE:** if you do not specify **all** existing fields in the specs for these two methods, they will overwrite any missing/unspecified (non-required) fields with null!!!

For that reason, we suggest you use `create()` to create new records and `merge()` to update existing records unless you wish to run the risk of overwriting fields with null.

##### Remove Method

Now if we want to delete the record that we've just created, we can use the `remove()` method.

In [ ]:
MyEntityTypeInstanceToDelete = c3.MyType.get('UniqueId')

In [ ]:
MyEntityTypeInstanceToDelete.remove()

##### Batching Methods

All the **CRUD** operations we demonstrated above can also be performed on a batch of data. To do so, simply leave out the `this` argument and pass in a list of UpsertSpecs/MergeSpecs.

In [ ]:
c3.MyType.createBatch([
                {"id": "UniqueId1",
                 "name": "Name1"},
                {"id": "UniqueId2",
                 "name": "Name2"},
                {"id": "UniqueId3",
                 "name": "Name3"}
    ])

In [ ]:
# view your updated data here
pd.DataFrame(c3.MyType.fetch().objs.toJson())

In [ ]:
c3.MyType.removeBatch([
                {"id": "UniqueId1",
                 "name": "Name1"},
                {"id": "UniqueId2",
                 "name": "Name2"},
                {"id": "UniqueId3",
                 "name": "Name3"}
    ])

In [ ]:
# view your updated data here
pd.DataFrame(c3.MyType.fetch().objs.toJson())

### Use This Space for "CHALLENGE: CRUD Operations" 
    
    Add additional cells as needed.

## 3. Prototype Methods in Python <a class="anchor" id="1.3">

### A. Write Code <a class="anchor" id="1.3A">

Provsion running code in package.

Declare Python methods in types with this syntax:

``` py
// Method takes myInput (a string) and returns a double
@py(env='myActionRuntime')
myPythonMethod: function(myInput: string): double
```

The .py file needs its own package imports.

### Use This Space for "CHALLENGE: Build Methods"
   

In [ ]:
import pandas as pd
import datetime as dt

def convert_dates_to_common_format(input_date):
        
        # if the date is a datetime, we need to strip timezone information
        if isinstance(input_date, dt.datetime):
            date = input_date.replace(tzinfo=None)
        # if the date is a string, we need to convert it to a datetime
        elif isinstance(input_date, str):
            date = dt.datetime.strptime(input_date, "%Y-%m-%dT%H:%M:%S")
        
        return date

def abs_day_diff(input_date_1, input_date_2):
    # Convert dates and subtract using the convert_dates_to_common_format funtion
    date_1 = convert_dates_to_common_format(input_date_1)
    date_2 = convert_dates_to_common_format(input_date_2)
    
    return abs((date_1 - date_2).days) / 365.

def lifeSpanInYears(bulbId):
    # Get startDate from SmartBulbMeasurement
    bulb = c3.SmartBulb.get(this={'id': bulbId})
    startDate = bulb.startDate
    
    # Get data from SmartBulbMeasurement
    limitSpec = -1
    defectFilter = "status == 1 && lumens == 0 && parent.id == '" + 'SBMS_serialNo_' + bulb.id + "'"
    smartbulbmeasurement_objs = c3.SmartBulbMeasurement.fetch(spec={'limit':limitSpec,
                                                                    'filter':defectFilter}).objs
    smartbulbmeasurement_pd = pd.DataFrame(smartbulbmeasurement_objs.toJson())
    
    # get endDate from filtered measurements
    smartbulbmeasurement_pd.sort_values(by='start',ascending=False)
    endDate = str(smartbulbmeasurement_pd[['start']].iloc[0].values[0])
    
    # return endDate - startDate from using abs_day_diff function
    return abs_day_diff(startDate, endDate)

In [ ]:
lifeSpanInYears('SMBLB1')

In [ ]:
c3.SmartBulb.lifeSpanInYears('SMBLB1')

For your challenge you will build one to two additional static methods:
-   `shortestLifeSpan: function(): string`  (**CHALLENGE**)

    Returns the string sentence: f"Bulb {bulbId} has the shortest lifespan = {lifespan} years"

    
-   `longestAndShortestLifespans: function(): string`  (**ADVANCED CHALLENGE**)

    Returns the string sentence: f'Longest Lifespan: {longest_lifespan_bulb_id} - {longest_lifespan_bulb}, Shortest Lifespan: {shortest_lifespan_bulb_id} - {shortest_lifespan_bulb}'



### Use This Space for "CHALLENGE: Build Methods" 
    
    Add additional cells as needed.

### Test Code <a class="anchor" id="1.3B">

After provisioning, test code from the static console or Jypyter

Use `c3.MyType.myPythonMethod(myInput)` to get the output of the function as it's been provisioned onto your tag.

### Use This Cell to Test that Your Code Has Provisioned And Is Available on the Platform
   

## 4. AsyncActions <a class="anchor" id="1.4">

The methods we just evaluated on the c3-server were executed in a synchronous fashion on a `C3 Master` server. We can just as easily call methods on types asynchronously and execute these methods on `C3 Worker` nodes.

In [ ]:
spec = c3.AsyncActionSpec(typeName='SmartBulb', 
                          action='lifeSpanInYears', 
                          args={'bulbId': 'SMBLB1'})

action = c3.AsyncAction.submit(spec)

In [ ]:
action.hasCompleted()

In [ ]:
action.get().result

In [ ]:
action.formatError()

Take a moment to piece together the data flow for the `AsyncAction` we just submitted. Questions to ask yourself:
- which node did this computation execute on?
- which c3 components were involved in making this computation possible?
- what kinds of computation should we distribute to `C3 Worker` nodes? Examples?

##### What happens if the action fails?

Note that the method lifeSpanInYears was defined for the SmartBulb Type. What happens if we try to run it on the Building Type?

In [ ]:
spec = c3.AsyncActionSpec(typeName='Building', 
                          action='lifeSpanInYears', 
                          args={'bulbId': 'SMBLB1'})

action = c3.AsyncAction.submit(spec)

In [ ]:
action.hasCompleted()

In [ ]:
action.formatError()

### Use This Space for "CHALLENGE: Execute AsyncActions"
   

# Make sure to SYNC your notebook to the server, then CLOSE AND HALT this notebook when you leave.
To sync: go to the File menu, Save and Checkpoint your notebook, and then select "Upload Notebook to C3.ai", or select the notebook in the tree view (check the box) and hit the "Sync" button.